## Note: This is a sample solution for the project. Projects will NOT be graded on the basis of how well the submission matches this sample solution. Projects will be graded on the basis of the rubric only.

# Background & Context

There is a huge demand for used cars in the Indian Market today. As sales of new cars have slowed down in the recent past, the pre-owned car market has continued to grow over the past years and is larger than the new car market now. Cars4U is a budding tech start-up that aims to find footholes in this market.

In 2018-19, while new car sales were recorded at 3.6 million units, around 4 million second-hand cars were bought and sold. There is a slowdown in new car sales and that could mean that the demand is shifting towards the pre-owned market. In fact, some car sellers replace their old cars with pre-owned cars instead of buying new ones. Unlike new cars, where price and supply are fairly deterministic and managed by OEMs (Original Equipment Manufacturer / except for dealership level discounts which come into play only in the last stage of the customer journey), used cars are very different beasts with huge uncertainty in both pricing and supply. Keeping this in mind, the pricing scheme of these used cars becomes important in order to grow in the market.

As a senior data scientist at Cars4U, you have to come up with a pricing model that can effectively predict the price of used cars and can help the business in devising profitable strategies using differential pricing. For example, if the business knows the market price, it will never sell anything below it. 

# Objective

* Explore and visualize the dataset.

* Build a linear regression model to predict the prices of used cars.

* Generate a set of insights and recommendations that will help the business.

**Data Dictionary -** 

S.No. : Serial Number

Name : Name of the car which includes Brand name and Model name

Location : The location in which the car is being sold or is available for purchase Cities

Year : Manufacturing year of the car

Kilometers_driven : The total kilometers driven in the car by the previous owner(s) in KM.

Fuel_Type : The type of fuel used by the car. (Petrol, Diesel, Electric, CNG, LPG)

Transmission : The type of transmission used by the car. (Automatic / Manual)

Owner : Type of ownership

Mileage : The standard mileage offered by the car company in kmpl or km/kg

Engine : The displacement volume of the engine in CC.

Power : The maximum power of the engine in bhp.

Seats : The number of seats in the car.

New_Price : The price of a new car of the same model in INR Lakhs.(1 Lakh = 100, 000)

Price : The price of the used car in INR Lakhs (1 Lakh = 100, 000)

# Loading libraries

In [1]:
# this will help in making the Python code more structured automatically (good coding practice)
%load_ext nb_black

ModuleNotFoundError: No module named 'nb_black'

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)

# to do mathematical computations
import math

# To build linear model for prediction
from sklearn.linear_model import LinearRegression

# To check model performance
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# to suppress warnings
import warnings

warnings.filterwarnings("ignore")

# Loading and exploring the data

Loading the data into python to explore and understand it.

In [ ]:
data = pd.read_csv("used_cars_data.csv")
print(f"There are {data.shape[0]} rows and {data.shape[1]} columns.")  # f-string

In [ ]:
# let's create a copy of the data
df = data.copy()

# let's view a sample of the data
np.random.seed(1)  # to get the same random results every time
df.sample(n=10)

`S.No.` is just an index for the data entry. In all likelihood, this column will not be a significant factor in determining the price of the car. 
Having said that, there are instances where the index of the data entry contains the information about time factor (an entry with a smaller index corresponds to data entered years ago). Therefore, we will not drop this variable just yet. Let us see if there is any relationship with the price when we do a bivariate analysis.

`Car names` contain a lot of model information. Let us check how many individual names we have. If they are too many, we can process this column to extract important information.

`Mileage`, `Engine`, and `Power` will also need some processing before we are able to explore them. We'll have to extract numerical information from these columns.

The `New_Price` column also needs some processing. This one also contains strings and a lot of missing values.

In [ ]:
df.info()

As expected, `Mileage`, `Engine`, `Power` and `New_Price` are objects when they should ideally be numerical. To be able to get summary statistics for these columns, We will have to process them first.

In [ ]:
df.duplicated().sum()

There are no duplicate values in the data.

# Processing Columns

Let us process `Mileage`, `Engine`, `Power`, and `New_Price` columns to extract numerical values from them.

#### 1. Mileage

We have car mileage in two units as per the data dictionary - kmpl and km/kg.

After a quick research on the internet, it is clear that these 2 units are used for cars of 2 different fuel types.

* kmpl (kilometers per litre) is used for petrol and diesel cars.
* km/kg (kilometers per kg) is used for CNG and LPG-based engines.

We have the variable `Fuel_type` in our data. Let us check if these observations hold true in our data also.

In [ ]:
df_mileage = df["Mileage"].str.split(" ", expand=True)
df_mileage.head()

In [ ]:
# let's verify that there are two units
df_mileage[1].value_counts()

In [ ]:
# we will create two new columns for mileage values and units
df["km_per_unit_fuel"] = df_mileage[0].astype(float)
df["mileage_unit"] = df_mileage[1]

# Checking the new dataframe
df.head()

In [ ]:
# Let's check if the units correspond to the fuel types
df.groupby(by=["Fuel_Type", "mileage_unit"]).size()

As expected, km/kg is for CNG/LPG cars and kmpl is for Petrol and Diesel cars.

#### 2. Engine 

As per the data dictionary, the `Engine` column indicates the displacement volume of the engine in CC.

Let's extract the numerical part of the column values.

In [ ]:
df_engine = df["Engine"].str.split(" ", expand=True)
df_engine.head()

In [ ]:
# let's verify that there is only one unit
df_engine[1].value_counts()

In [ ]:
# we will create a new column for engine values
df["engine_num"] = df_engine[0].astype(float)

# Checking the new dataframe
df.head()

#### 3. Power 

As per the data dictionary, the `Power` column indicates the maximum power of the engine in bhp.

Let's extract the numerical part of the column values.

In [ ]:
df_power = df["Power"].str.split(" ", expand=True)
df_power.head()

In [ ]:
# let's verify that there is only one unit
df_power[1].value_counts()

In [ ]:
# we will create a new column for power values
df["power_num"] = df_power[0].astype(float)

# Checking the new dataframe
df.head()

The error occurred as there are some non-numeric values like 'null' in the numeric part of the original `Power` column.

In [ ]:
# let's check the values where there is 'null'
df_power[df_power[0] == "null"]

In [ ]:
# let's define a function to convert the Power column to float


def power_to_num(power_val):
    """
    This function takes in a string representing he maximum power of the engine in bhp
    and converts it to a number. For example, '126.2 bhp' becomes 126.2
    If the input is already numeric, which probably means it's NaN,
    this function just returns np.nan.
    """
    if isinstance(power_val, str):  # checks if `power_val` is a string
        if power_val.startswith("null"):  # checks if `power_val` starts with 'null'
            return np.nan
        elif power_val.endswith("bhp"):
            return float(power_val.replace(" bhp", ""))
    else:  # this happens when the power is np.nan
        return np.nan

In [ ]:
# let's apply the function to the Power column
df["power_num"] = df["Power"].apply(power_to_num)

# Checking the new dataframe
df.head()

#### 4. New_Price 

We know that `New_Price` is the price of a new car of the same model in INR Lakhs (1 Lakh INR = 100, 000 INR)

Let's extract the numerical part of the column values.

In [ ]:
df_new_price = df["New_Price"].str.split(" ", expand=True)
df_new_price.head()

In [ ]:
# let's verify that there is only one unit
df_new_price[1].value_counts()

* There are two units - Lakh and Cr.
* 1 Cr = 100 Lakhs.

In [ ]:
# let's define a function to convert the New_Price column to float


def new_price_to_num(new_price_val):
    """
    This function takes in a string representing a new car price
    and converts it to a number. For example, '8.61 Lakh' becomes 8.61.
    If the input is already numeric, which probably means it's NaN,
    this function just returns np.nan.
    """
    if isinstance(new_price_val, str):  # checks if `new_price_val` is a string
        multiplier = 1  # handles Lakh vs Cr values
        if new_price_val.endswith("Lakh"):
            multiplier = 1
        elif new_price_val.endswith("Cr"):
            multiplier = 100
        return float(new_price_val.replace(" Lakh", "").replace(" Cr", "")) * multiplier
    else:  # this happens when the current new_price is np.nan
        return np.nan

In [ ]:
# let's apply the function to the New_Price column
df["new_price_num"] = df["New_Price"].apply(new_price_to_num)

# Checking the new dataframe
df.head()

# Feature Engineering

The `Name` column in the current format might not be very useful in our analysis.
Since the name contains both the brand name and the model name of the vehicle, the column would have too many unique values to be useful in prediction.

In [ ]:
df["Name"].nunique()

With 2041 unique names, car names are not going to be great predictors of the price in our current data.

But we can process this column to extract important information and see if that reduces the number of levels for this information.

#### 1. Car Brand Name

In [ ]:
# Extract Brand Names
df["Brand"] = df["Name"].apply(lambda x: x.split(" ")[0].lower())

# Check the data
df["Brand"].value_counts()

In [ ]:
plt.figure(figsize=(15, 7))
sns.countplot(y="Brand", data=df, order=df["Brand"].value_counts().index)

#### 2. Car Model Name

In [ ]:
# Extract Model Names
df["Model"] = df["Name"].apply(lambda x: x.split(" ")[1].lower())

# Check the data
df["Model"].value_counts()

In [ ]:
plt.figure(figsize=(15, 7))
sns.countplot(y="Model", data=df, order=df["Model"].value_counts().index[:30])

It is clear from the above charts that our dataset contains used cars from luxury as well as budget-friendly brands.

We can create a new variable using this information. We will bin all our cars into the following 3 categories later:

1. Budget-Friendly
2. Mid Range
3. Luxury Cars

#### 3. Car_category

In [ ]:
df.groupby(["Brand"])["Price"].mean().sort_values(ascending=False)

The output is very close to our expectation (domain knowledge), in terms of brand order. The mean price of a used Lamborghini is 120 Lakhs and that of cars from other luxury brands follow in descending order.

Towards the bottom end, we have the more budget-friendly brands.

We can see that there is some missingness in our data. Let us come back to creating this variable once we have removed missingness from the data.

# Exploratory Data Analysis

In [ ]:
# Basic summary stats - Numeric variables
df.describe().T

**Observations**

1. S.No. clearly has no interpretation here but as discussed earlier let us drop it only after having looked at the initial linear model.
2. Kilometers_Driven values have an incredibly high range. We should check a few of the extreme values to get a sense of the data.
3. Minimum and maximum number of seats in the car also warrant a quick check. On average, a car seems to have 5 seats, which is about right.
4. We have used cars being sold at less than a lakh rupees and as high as 160 lakh, as we saw for Lamborghini earlier. We might have to drop some of these outliers to build a robust model.
5. The minimum mileage being 0 is also concerning, we'll have to check what is going on.
6. Engine and Power mean and median values are not very different. Only someone with more domain knowledge would be able to comment further on these attributes.
7. The new price range seems right. We have both budget-friendly Maruti cars and Lamborghinis in our stock. Mean being twice that of the median suggests that there are only a few very high range brands, which again makes sense.

In [ ]:
# checking extreme values in Kilometers_Driven
df.sort_values(by=["Kilometers_Driven"], ascending=False).head(10)

It looks like the first row here is a data entry error. A car manufactured as recently as 2017 having been driven 6500000 km is almost impossible.

The other observations that follow are also on a higher end. There is a good chance that these are outliers. We'll look at this further while doing the univariate analysis.

In [ ]:
# checking extreme values in Kilometers_Driven
df.sort_values(by=["Kilometers_Driven"], ascending=True).head(10)

After looking at the columns `Year`, `New_Price`, and `Price`, these entries seem feasible.

1000 might be the default value in this case. Quite a few cars having driven exactly 1000 km is suspicious.

In [ ]:
# checking extreme values in Seats
df.sort_values(by=["Seats"], ascending=True).head(5)

Audi A4 having 0 seats is clearly a data entry error. This column warrants some outlier treatment, or we can treat Seats = 0 as a missing value. Overall, there doesn't seem not much to be concerned about here.  

In [ ]:
# Let us check if we have a similar car in our dataset.
df[df["Name"].str.startswith("Audi A4")]

Looks like an Audi A4 typically has 5 seats.

In [ ]:
# Let us replace #seats in row index 3999 form 0 to 5
df.loc[3999, "Seats"] = 5.0

In [ ]:
# checking extreme values in Seats
df.sort_values(by=["Seats"], ascending=False).head(5)

Of course, a Toyota Qualis has 10 seats and so does a Tata Sumo. We don't see any data entry error here.

In [ ]:
# checking extreme values in Mileage - km_per_unit_fuel
df.sort_values(by=["km_per_unit_fuel"], ascending=True).head(10)

We will have to treat Mileage = 0 as missing values

In [ ]:
# checking extreme values in Mileage - km_per_unit_fuel
df.sort_values(by=["km_per_unit_fuel"], ascending=False).head(10)

Maruti Wagon R and Maruti Alto CNG versions are budget-friendly cars with high mileage, so these data points are fine.

In [ ]:
# looking at value counts for non-numeric features

num_to_display = 10  # defining this up here so it's easy to change later
for colname in df.dtypes[df.dtypes == "object"].index:
    val_counts = df[colname].value_counts(dropna=False)  # will also show the NA counts
    print(val_counts[:num_to_display])
    if len(val_counts) > num_to_display:
        print(f"Only displaying first {num_to_display} of {len(val_counts)} values.")
    print("\n\n")  # just for more space in between

Since we haven't dropped the original columns that we processed, we have a few redundant outputs here.

We had checked cars of different `Fuel_Type` earlier, but we did not encounter the 2 electric cars. Let us check why.

In [ ]:
df.loc[df["Fuel_Type"] == "Electric"]

Mileage values for these cars are NaN, that is why we did not encounter these earlier with groupby.

Electric cars are very new in the market and very rare in our dataset. We can consider dropping these two observations if they turn out to be outliers later. There is a good chance that we will not be able to create a good price prediction model for electric cars, with the currently available data.

New Price for 6247 entries is missing. We need to explore if we can impute these or we should drop this column altogether.

## Missing Values

Before we start looking at the individual distributions and interactions, let's quickly check the missingness in the data.

In [ ]:
df.isnull().sum()

* 2 Electric car variants don't have entries for Mileage.
* Engine displacement information of 46 observations is missing and the maximum power of 175 entries is missing.
* Information about the number of seats is not available for 53 entries.
* New price, as we saw earlier, has a huge missing count. We'll have to see if there is a pattern here.
* Price is also missing for 1234 entries. Since price is the response variable that we want to predict, we will have to drop these rows when we actually build a model. These rows will not be able to help us in modeling or model evaluation. But while we are analyzing the distributions and doing missing value imputations, we will keep using information from these rows.

In [ ]:
# dropping the redundant columns.
df.drop(
    columns=["Mileage", "mileage_unit", "Engine", "Power", "New_Price"], inplace=True
)

## Distributions

### Price

In [ ]:
sns.histplot(data=df, x="Price", kde=True)

This is a highly skewed distribution. Let us use log transformation on this column to see if that helps normalize the distribution.

In [ ]:
sns.histplot(np.log(df["Price"]), kde=True)
plt.xlabel("Log(price)")

Using the log transformation has definitely helped in reducing the skew.

In [ ]:
# Creating a new column with the transformed variable.
df["price_log"] = np.log(df["Price"])

### Price vs Location

In [ ]:
plt.figure(figsize=(15, 7))
sns.boxplot(x="Location", y="Price", data=df)

The price of used cars has a large IQR in Coimbatore and Bangalore

### Kilometers_Driven

In [ ]:
sns.histplot(data=df, x="Kilometers_Driven", kde=True)

In [ ]:
# Log transformation
sns.histplot(np.log(df["Kilometers_Driven"]), kde=True)
plt.xlabel("Log(Kilometers_Driven)")

Transformation has reduced the extreme skewness.

In [ ]:
df["kilometers_driven_log"] = np.log(df["Kilometers_Driven"])

### Bivariate Distributions

In [ ]:
sns.pairplot(data=df, hue="Fuel_Type")

Zooming into these plots gives us a lot of information.

* Contrary to intuition, Kilometers Driven does not seem to have a relationship with the price.
* Price has a positive relationship with Year. Newer the car, the higher the price.
* S.No. does not capture any information that we were hoping for. The temporal element of variation is captured in the year column.
* 2 seater cars are all luxury variants. Cars with 8-10 seats are exclusively mid to high range.
* Mileage does not seem to show much relationship with the price of used cars.
* Engine displacement and Power of the car have a positive relationship with the price.
* New Price and Used Car Price are also positively correlated, which is expected.
* Kilometers Driven has a peculiar relationship with the Year variable. Generally, the newer the car lesser the distance it has traveled, but this is not always true.
* CNG cars are conspicuous outliers when it comes to Mileage. The mileage of these cars is very high.
* The mileage and power of newer cars are increasing owing to advancements in technology.
* Mileage has a negative correlation with engine displacement and power. More powerful the engine, the more fuel it consumes in general.

### Correlation between numerical variables

In [ ]:
plt.figure(figsize=(12, 7))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")

* Power and engine are important predictors of price
* We will have to work on imputing New Price missing values because this is a very important feature in predicting used car price accurately 

## Missing Value Treatment

In [ ]:
# let's check again for missing values
df.isnull().sum()

### Seats

In [ ]:
# let's look at a few rows where seats is missing
df[df["Seats"].isnull()]

We'll impute these missing values one-by-one by taking the median number of seats for the particular car using the Brand and Model name.

In [ ]:
df.groupby(["Brand", "Model"], as_index=False)["Seats"].median()

In [ ]:
# Impute missing Seats
df["Seats"] = df.groupby(["Brand", "Model"])["Seats"].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
# Check 'Seats'
df[df["Seats"].isnull()]

In [ ]:
# Maruti Estilo can accomodate 5
df["Seats"] = df["Seats"].fillna(5.0)

We will use a similar method to fill missing values for engine, power, and new price.

In [ ]:
df["engine_num"] = df.groupby(["Brand", "Model"])["engine_num"].transform(
    lambda x: x.fillna(x.median())
)
df["power_num"] = df.groupby(["Brand", "Model"])["power_num"].transform(
    lambda x: x.fillna(x.median())
)
df["new_price_num"] = df.groupby(["Brand", "Model"])["new_price_num"].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
df.isnull().sum()

* There are still some missing values in power, mileage and new_price_num.
* There are a few car brands and models in our dataset that do not contain the new price information at all.
* We'll impute these missing values one-by-one by taking the median number of seats for the particular car using the Brand name.

In [ ]:
cols1 = ["power_num", "km_per_unit_fuel", "new_price_num"]

for ii in cols1:
    df[ii] = df.groupby(["Brand"])[ii].transform(lambda x: x.fillna(x.median()))

In [ ]:
df.isnull().sum()

* There are still some missing values in power and new_price_num.
* We'll have to estimate the new price using median of the data.

In [ ]:
cols1 = ["power_num", "km_per_unit_fuel", "new_price_num"]

for ii in cols1:
    df[ii] = df[ii].fillna(df[ii].median())

In [ ]:
df.isnull().sum()

In [ ]:
# dropping the redundant columns
df.drop(columns=["Kilometers_Driven", "Name", "S.No."], inplace=True)

# dropping the rows where 'Price' == NaN, and proceed to modeling
df = df[df["Price"].notna()]

# Linear Model Building

1. We want to predict the price, so we will use the normalized version 'price_log' for modeling.
2. Before we proceed to build a model, we'll have to encode categorical features. We will drop categorical features like Name.
3. We'll split the data into train and test to be able to evaluate the model that we build on the train data.
4. We will build a Linear Regression model using the train data and evaluate the model performance.

### Define dependent variable

In [ ]:
ind_vars = df.drop(["Price", "price_log"], axis=1)
dep_var = df["price_log"]

### Creating dummy variables

In [ ]:
def encode_cat_vars(x):
    x = pd.get_dummies(
        x,
        columns=x.select_dtypes(include=["object", "category"]).columns.tolist(),
        drop_first=True,
    )
    return x


ind_vars_num = encode_cat_vars(ind_vars)
ind_vars_num.head()

### Split the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    ind_vars_num, dep_var, test_size=0.3, random_state=1
)

In [ ]:
print("Number of rows in train data =", x_train.shape[0])
print("Number of rows in train data =", x_test.shape[0])

### Fitting a linear model

In [ ]:
lin_reg_model = LinearRegression()
lin_reg_model.fit(x_train, y_train)

In [ ]:
# let us check the coefficients and intercept of the model

coef_df = pd.DataFrame(
    np.append(lin_reg_model.coef_, lin_reg_model.intercept_),
    index=x_train.columns.tolist() + ["Intercept"],
    columns=["Coefficients"],
)
coef_df

**Let us check the model performance on training data.**

In [ ]:
# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# Model performance check
def model_perf(model, inp, out):

    y_pred = np.exp(model.predict(inp))
    y_act = np.exp(out)

    return pd.DataFrame(
        {
            "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
            "MAE": mean_absolute_error(y_act, y_pred),
            "R^2": r2_score(y_act, y_pred),
            "Adjusted R^2": adj_r2(inp, y_act, y_pred),
        },
        index=[0],
    )

In [ ]:
# Checking model performance on train set
print("Training Performance:")
model_perf(lin_reg_model, x_train, y_train)

* Both the R-squared and Adjusted R squared of our model are very high. This is a clear indication that we have been able to create a very good model that is able to explain variance in the price of used cars up to 94%.
* The model is not an underfitting model.
* Let us do a quick performance check on the test data.

In [ ]:
# Checking model performance on test set
print("Test Performance:")
model_perf(lin_reg_model, x_test, y_test)

* The model can explain more than 91% of the variation in the test data, which is very good.
* Root Mean Squared Error of train and test data are close, which indicates that our model is not overfitting the train data.
* Mean Absolute Error indicates that our current model is able to predict used car prices within a mean error of 1.3 lakhs on test data.
* The units of both RMSE and MAE are the same - Lakhs in this case. But RMSE is greater than MAE because it penalizes the outliers more.

## Forward Feature Selection

Let us try using forward feature selection to check if we can get a good model performance using lesser number of features.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

reg = LinearRegression()

# Build step forward feature selection
sfs = SFS(
    reg,
    k_features=x_train.shape[1],
    forward=True,  # k_features denotes "Number of features to select"
    floating=False,
    scoring="r2",
    verbose=2,
    n_jobs=-1,  # this will ensure all CPU cores are being used for computation
    cv=5,
)

# Perform SFFS
sfs = sfs.fit(x_train, y_train)

In [ ]:
# to plot the performance with addition of each feature
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig1 = plot_sfs(sfs.get_metric_dict(), kind="std_err", figsize=(15, 5))
plt.title("Sequential Forward Selection (w. StdErr)")
plt.xticks(
    np.arange(0, 264, 10), np.arange(0, 264, 10), rotation=90
)  # to make the tick marks readable
plt.show()

* The model performance appears to have become constant somewhere around 118 features and then starts falling around 239 features.
* The improvement in performance from 85 to 118 features is not that high either.
* We will take 85 as the number of features to consider.

In [ ]:
reg = LinearRegression()

# # Build step forward feature selection
sfs = SFS(
    reg,
    k_features=85,
    forward=True,
    floating=False,
    scoring="r2",
    verbose=2,
    cv=5,
    n_jobs=-1,
)

# Perform SFFS
sfs = sfs.fit(x_train, y_train)

In [ ]:
# Which features are important?
feat_cols = list(sfs.k_feature_idx_)
print(feat_cols)

In [ ]:
x_train.columns[feat_cols]

**Now we will fit a sklearn model using these features only.**

In [ ]:
x_train2 = x_train[x_train.columns[feat_cols]]

In [ ]:
# Creating new x_test with the same 20 variables that we selected for x_train
x_test2 = x_test[x_train2.columns]

In [ ]:
# Fitting linear model
lin_reg_model2 = LinearRegression()
lin_reg_model2.fit(x_train2, y_train)

# let us check the coefficients and intercept of the model

coef_df = pd.DataFrame(
    np.append(lin_reg_model2.coef_, lin_reg_model2.intercept_.flatten()),
    index=x_train2.columns.tolist() + ["Intercept"],
    columns=["Coefficients"],
)
coef_df

In [ ]:
# model performance on train set
print("Training Performance:")
model_perf(lin_reg_model2, x_train2, y_train)

In [ ]:
# model performance on test set
print("Test Performance:")
model_perf(lin_reg_model2, x_test2, y_test)

**We have been able to explain more than 92% of the variation in the test data using less than one-third the number of features used in the previous model.**

## Observations

1. With our linear regression model, we have been able to capture more than 92% of the variation in the test data.


2. Mean Absolute Error indicates that our current model is able to predict used car prices within a mean error of approx. 1.4 lakhs on the test data.


3. Factors like the year of manufacture, numbers of seats, maximum power of the engine, few locations, few brands, etc. tend to increase the price of the used car.


4. Factors like distance covered in unit fuel, log of the number of kilometers drive, few locations, few brands, etc. tend to decrease the price of the used car.

## Business Insights and Recommendations

* Cars with a lesser number of kilometers driven should be preferred. 


* Some markets tend to have higher prices. Cars4U should focus more on these markets, and set up offices in these areas if needed.


* We will have to analyze the cost side of things before we can talk about profitability in the business. We should gather data regarding that.


* The next step post that would be to cluster different sets of data and see if we should make multiple models for different locations/car types.

## Add-on: Analysing predictions where we were way off the mark

In [ ]:
# Extracting the rows from original data frame df where indexes are same as the training data
original_df = df[df.index.isin(x_train2.index.values)].copy()

# Extracting predicted values and residuals from the final model
fitted_values = lin_reg_model2.predict(x_train2)
residuals = fitted_values - y_train

# Add new columns for predicted values
original_df["Predicted price_log "] = fitted_values
original_df["Predicted Price"] = np.exp(fitted_values)
original_df["residuals"] = residuals
original_df["Abs_residuals"] = np.exp(residuals)
original_df["Difference in Lakhs"] = np.abs(
    original_df["Price"] - original_df["Predicted Price"]
)

# Let us look at the top 50 predictions where our model made highest extimation errors (on train data)
original_df.sort_values(by=["Difference in Lakhs"], ascending=False).head(50)

* A 2017 Land Rover, whose new model sells at 230 Lakhs and the used version sold at 160 Lakhs was predicted to be sold at < 3 Lakhs. It is not apparent after looking at numerical predictors, why our model predicted such low value here. This could be because many other land rovers in our data seems to have sold at lower prices.
* Another entry in the list here is a Lamborghini Gallardo that was sold at 120 Lakhs but our model predicted the price around 3 lakhs. This is a huge error by the model. However, there might be a data entry error here as the price of a new Gallardo is set at around 11 Lakhs, which is less than the selling price of a used Gallardo.
* There are a few instances where the model predicts lesser than the actual selling price. These could be a cause for concern. The model predicting lesser than potential selling price is not good for business.

Let us quickly visualise some of these observations. 

In [ ]:
sns.scatterplot(
    data=original_df,
    x="Difference in Lakhs",
    y="Price",
    hue=original_df["Fuel_Type"],
)

Our model predicts that resale value of diesel cars is higher compared to petrol cars.

## Add-on: Advanced Data Preprocessing

In [ ]:
df = data.copy()

#### 1. Mileage

We have car mileage in two units as per the data dictionary - kmpl and km/kg.

After a quick research on the internet, it is clear that these 2 units are used for cars of 2 different fuel types.

* kmpl (kilometers per litre) is used for petrol and diesel cars.
* km/kg (kilometers per kg) is used for CNG and LPG-based engines.

We have the variable `Fuel_type` in our data. Let us check if these observations hold true in our data also.

In [ ]:
# Create 2 new columns after splitting the mileage values.
km_per_unit_fuel = []
mileage_unit = []

for observation in df["Mileage"]:
    if isinstance(observation, str):
        if (
            observation.split(" ")[0]
            .replace(".", "", 1)
            .isdigit()  # first element should be numeric
            and " " in observation  # space between numeric and unit
            and (
                observation.split(" ")[1]
                == "kmpl"  # units are limited to "kmpl" and "km/kg"
                or observation.split(" ")[1] == "km/kg"
            )
        ):
            km_per_unit_fuel.append(float(observation.split(" ")[0]))
            mileage_unit.append(observation.split(" ")[1])
        else:
            # To detect if there are any observations in the column that do not follow
            # the expected format [number + ' ' + 'kmpl' or 'km/kg']
            print(
                "The data needs further processing. All values are not similar ",
                observation,
            )
    else:
        # If there are any missing values in the mileage column,
        # we add corresponding missing values to the 2 new columns
        km_per_unit_fuel.append(np.nan)
        mileage_unit.append(np.nan)

No print output from the function above. The values are all in the expected format or NaNs.

In [ ]:
# Add the new columns to the data

df["km_per_unit_fuel"] = km_per_unit_fuel
df["mileage_unit"] = mileage_unit

# Checking the new dataframe
df.head(5)

In [ ]:
# Let us check if the units correspond to the fuel types
df.groupby(by=["Fuel_Type", "mileage_unit"]).size()

As expected, km/kg is for CNG/LPG cars and kmpl is for Petrol and Diesel cars.

#### 2. Engine 

As per the data dictionary, the `Engine` column indicates the displacement volume of the engine in CC.

We will make sure that all the observations follow the same format - [numeric + " " + "CC"] and create a new numeric column from this column.

This time, let's use a regex to make all the necessary checks.

In [ ]:
# re module provides support for regular expressions
import re

# Create a new column after splitting the engine values.
engine_num = []

# Regex for numeric + " " + "CC"  format
regex_engine = "^\d+(\.\d+)? CC$"

for observation in df["Engine"]:
    if isinstance(observation, str):
        if re.match(regex_engine, observation):
            engine_num.append(float(observation.split(" ")[0]))
        else:
            # To detect if there are any observations in the column that do not follow [numeric + " " + "CC"]  format
            print(
                "The data needs furthur processing. All values are not similar ",
                observation,
            )
    else:
        # If there are any missing values in the engine column, we add missing values to the new column
        engine_num.append(np.nan)

No print output from the function above. The values are all in the same format - [numeric + " " + "CC"] or NaNs.

In [ ]:
# Add the new column to the data

df["engine_num"] = engine_num

# Checking the new dataframe
df.head(5)

#### 3. Power 

As per the data dictionary, the `Power` column indicates the maximum power of the engine in bhp.

We will make sure that all the observations follow the same format - [numeric + " " + "bhp"] and create a new numeric column from this column, like we did for the `Engine` column.

In [ ]:
# Create a new column after splitting the power values.
power_num = []

# Regex for numeric + " " + "bhp"  format
regex_power = "^\d+(\.\d+)? bhp$"

for observation in df["Power"]:
    if isinstance(observation, str):
        if re.match(regex_power, observation):
            power_num.append(float(observation.split(" ")[0]))
        else:
            # To detect if there are any observations in the column that do not follow [numeric + " " + "bhp"]  format
            # that we see in the sample output
            print(
                "The data needs furthur processing. All values are not similar ",
                observation,
            )
    else:
        # If there are any missing values in the power column, we add missing values to the new column
        power_num.append(np.nan)

We can see that some Null values in power column exist as 'null bhp' string.
Let us replace these with NaNs

In [ ]:
power_num = []

for observation in df["Power"]:
    if isinstance(observation, str):
        if re.match(regex_power, observation):
            power_num.append(float(observation.split(" ")[0]))
        else:
            power_num.append(np.nan)
    else:
        # If there are any missing values in the power column, we add missing values to the new column
        power_num.append(np.nan)

# Add the new column to the data
df["power_num"] = power_num

# Checking the new dataframe
df.head(10)

#### 4. New_Price 

We know that `New_Price` is the price of a new car of the same model in INR Lakhs.(1 Lakh INR = 100,000 INR)

This column clearly has a lot of missing values. We will impute the missing values later. For now we will only extract the numeric values from this column.

In [ ]:
# Create a new column after splitting the New_Price values.
new_price_num = []

# Regex for numeric + " " + "Lakh"  format
regex_power = "^\d+(\.\d+)? Lakh$"

for observation in df["New_Price"]:
    if isinstance(observation, str):
        if re.match(regex_power, observation):
            new_price_num.append(float(observation.split(" ")[0]))
        else:
            # To detect if there are any observations in the column that do not follow [numeric + " " + "Lakh"]  format
            # that we see in the sample output
            print(
                "The data needs furthur processing. All values are not similar ",
                observation,
            )
    else:
        # If there are any missing values in the New_Price column, we add missing values to the new column
        new_price_num.append(np.nan)

* Not all values are in Lakhs.

* There are a few observations that are in Crores as well

* Let us convert these to lakhs using the conversion rate 1 Cr = 100 Lakhs.

In [ ]:
new_price_num = []

for observation in df["New_Price"]:
    if isinstance(observation, str):
        if re.match(regex_power, observation):
            new_price_num.append(float(observation.split(" ")[0]))
        else:
            # Converting values in Crore to lakhs
            new_price_num.append(float(observation.split(" ")[0]) * 100)
    else:
        # If there are any missing values in the New_Price column, we add missing values to the new column
        new_price_num.append(np.nan)

# Add the new column to the data
df["new_price_num"] = new_price_num

# Checking the new dataframe
df.head(5)